# Here is a title
here is some text

In [4]:
import tensorflow as tf

c:\Users\matthias\miniconda3\envs\thunder_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\matthias\miniconda3\envs\thunder_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\matthias\miniconda3\envs\thunder_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\matthias\miniconda3\envs\thunder_e

In [5]:
print('Well, hello there!')

Well, hello there!


In [6]:
def some_fun (fun_1):
    """This function prints a variable
    

    Args:
        fun_1 (string): a string to be printed 
    """
    print(fun_1)

In [7]:
this_is_fun = 'FUN'
some_fun(this_is_fun)

FUN
